In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.kalman import MerweScaledSigmaPoints
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from filterpy.monte_carlo import systematic_resample
import scipy.stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from scipy import stats
import time

In [ ]:
# Metin dosyasını okuma ve işleme
data_path = "CoolTerm Capture (Untitled_0.stc) 2024-05-03 14-00-24-140.txt"

with open(data_path, 'r') as file:
    lines = file.readlines()

processed_data = []
temp_data = {}

In [ ]:
# Veri işleme
for line in lines:
    line = line.strip()
    if 'Latitude:' in line:
        temp_data['Latitude'] = float(line.split(':')[1].strip())
    elif 'Longitude:' in line:
        temp_data['Longitude'] = float(line.split(':')[1].strip())
    elif 'Speed (km/s):' in line:
        temp_data['Speed'] = float(line.split(':')[1].strip())
    elif 'AccelX:' in line:
        parts = line.split(',')
        temp_data['AccelX'] = float(parts[0].split(':')[1].strip())
        temp_data['AccelY'] = float(parts[1].split(':')[1].strip())
        temp_data['AccelZ'] = float(parts[2].split(':')[1].strip())
        temp_data['GyroX'] = float(parts[3].split(':')[1].strip())
        temp_data['GyroY'] = float(parts[4].split(':')[1].strip())
        temp_data['GyroZ'] = float(parts[5].split(':')[1].strip())
        # Her bir GPS okumasından sonra mevcut veriyi listeye ekle
        if 'Latitude' in temp_data and 'Longitude' in temp_data and 'Speed' in temp_data:
            processed_data.append(temp_data.copy())  # Yeni bir veri grubu için temp_data'yı kopyala

In [ ]:
# DataFrame oluştur
df = pd.DataFrame(processed_data)

# Verilerin ilk birkaç satırını yazdır
print(df.head())

# DataFrame'i CSV olarak kaydet
df.to_csv('processed_data_complete.csv', index=False)

# Başlangıç tarihi ve saati dosya adından alınır
start_datetime = pd.to_datetime('2024-05-03 14:00:24.140')

# Zaman damgası sütunu oluşturma
df['Timestamp'] = [start_datetime + pd.to_timedelta(1 * i, unit='s') for i in range(len(df))]

# Güncellenmiş DataFrame'i CSV olarak kaydet
df.to_csv('processed_data_with_timestamps.csv', index=False)

In [ ]:
# Verileri yükle
df = pd.read_csv('processed_data_with_timestamps.csv')

# Basit bir sensör füzyon örneği: Hız ve ivmelenme verilerini kullanarak kinetik enerji hesaplayalım
# Kinetik enerji = 1/2 * m * v^2 (m: kütle, v: hız (m/s cinsinden))
# Örnek kütle değeri (araba yaklaşık olarak): 6500 kg varsayalım

In [ ]:
m = 6500  # kg

# Hızı m/s cinsine dönüştür (km/s'den m/s'ye çevirme)
df['Speed_m/s'] = df['Speed'] * 1000 / 3600

# Kinetik Enerji hesaplama
df['Kinetic_Energy'] = 0.5 * m * df['Speed_m/s'] ** 2

# İvmeölçer verilerinden toplam ivme büyüklüğü hesaplayalım
df['Total_Acceleration'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

In [ ]:
# Verileri görselleştirelim
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(df['Timestamp'], df['Kinetic_Energy'], label='Kinetic Energy')
plt.xlabel('Time')
plt.ylabel('Energy (Joules)')
plt.title('Kinetic Energy Over Time')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(df['Timestamp'], df['Total_Acceleration'], label='Total Acceleration', color='red')
plt.xlabel('Time')
plt.ylabel('Acceleration (m/s^2)')
plt.title('Total Acceleration Over Time')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Sonuçları kaydet
df.to_csv('C:/Users/yigitc/Desktop/sensor_fusion/processed_data_with_timestamps.csv', index=False)

In [ ]:
# Load data
data = pd.read_csv('C:/Users/yigitc/Desktop/sensor_fusion/processed_data_with_timestamps.csv')

In [ ]:
# Extract relevant columns
accel = data[['AccelX', 'AccelY', 'AccelZ']].values
gyro = data[['GyroX', 'GyroY', 'GyroZ']].values
gps = data[['Latitude', 'Longitude', 'Speed']].values
timestamps = pd.to_datetime(data['Timestamp'])

In [ ]:
# Normalize data (example normalization, adjust as needed)
accel = (accel - np.mean(accel, axis=0)) / np.std(accel, axis=0)
gyro = (gyro - np.mean(gyro, axis=0)) / np.std(gyro, axis=0)

In [ ]:
# Initialize EKF parameters
state = np.zeros(12)  # [x, y, z, vx, vy, vz, ax, ay, az, gx, gy, gz]
covariance = np.eye(12) * 0.1
process_noise = np.eye(12) * 0.1
measurement_noise_accel = np.eye(3) * 0.1
measurement_noise_gyro = np.eye(3) * 0.1
measurement_noise_gps = np.eye(3) * 0.5

In [ ]:
# State transition model
def state_transition(state, dt):
    F = np.eye(12)
    F[0, 3] = F[1, 4] = F[2, 5] = dt
    F[3, 6] = F[4, 7] = F[5, 8] = dt
    F[6, 9] = F[7, 10] = F[8, 11] = dt
    return F @ state

In [ ]:
# Measurement models
def measurement_model_accel(state):
    return state[6:9]

In [ ]:
def measurement_model_gyro(state):
    return state[9:12]

In [ ]:
def measurement_model_gps(state):
    return state[0:3]

In [ ]:
# Predict step
def predict(state, covariance, process_noise, dt):
    state = state_transition(state, dt)
    F = np.eye(12)
    F[0, 3] = F[1, 4] = F[2, 5] = dt
    F[3, 6] = F[4, 7] = F[5, 8] = dt
    F[6, 9] = F[7, 10] = F[8, 11] = dt
    covariance = F @ covariance @ F.T + process_noise
    return state, covariance

In [ ]:
# Update step
def update(state, covariance, measurement, measurement_model, measurement_noise):
    H = np.zeros((3, 12))
    if measurement_model == measurement_model_accel:
        H[:, 6:9] = np.eye(3)
    elif measurement_model == measurement_model_gyro:
        H[:, 9:12] = np.eye(3)
    elif measurement_model == measurement_model_gps:
        H[:, 0:3] = np.eye(3)
    y = measurement - measurement_model(state)
    S = H @ covariance @ H.T + measurement_noise
    K = covariance @ H.T @ np.linalg.inv(S)
    state = state + K @ y
    covariance = (np.eye(len(state)) - K @ H) @ covariance
    return state, covariance

In [ ]:
# Sensor fusion using EKF
fused_data_ekf = []
for i in range(1,len(data)):
    dt = (timestamps[i] - timestamps[i - 1]).total_seconds()
    state, covariance = predict(state, covariance, process_noise, dt)
    state, covariance = update(state, covariance, accel[i], measurement_model_accel, measurement_noise_accel)
    state, covariance = update(state, covariance, gyro[i], measurement_model_gyro, measurement_noise_gyro)
    if not np.isnan(gps[i]).any():
        state, covariance = update(state, covariance, gps[i], measurement_model_gps, measurement_noise_gps)
    fused_data_ekf.append(state.copy())

In [ ]:
# Convert fused data to DataFrame
fused_data_ekf = pd.DataFrame(fused_data_ekf, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'ax', 'ay', 'az', 'gx', 'gy', 'gz'])


In [ ]:
# Plot results
plt.figure(figsize=(10, 6))
plt.subplot(311)
plt.plot(fused_data_ekf['x'], label='EKF X Position')
plt.legend()
plt.subplot(312)
plt.plot(fused_data_ekf['y'], label='EKF Y Position')
plt.legend()
plt.subplot(313)
plt.plot(fused_data_ekf['z'], label='EKF Z Position')
plt.legend()
plt.show()

# Doğrulama

In [ ]:
# İlk birkaç satırı görüntüleme
print(df.head())

# Verinin genel istatistiklerini kontrol etme
print(df.describe())


In [ ]:
#pip install filterpy

In [ ]:
# Durum geçiş ve ölçüm modelleri
def fx(state, dt):
    F = np.eye(12)
    F[0, 3] = F[1, 4] = F[2, 5] = dt
    F[3, 6] = F[4, 7] = F[5, 8] = dt
    F[6, 9] = F[7, 10] = F[8, 11] = dt
    return F @ state

def hx(state):
    return np.hstack((state[:3], state[6:]))

def resample(weights):
    n = len(weights)
    indices = []
    C = [0.] + [sum(weights[:i+1]) for i in range(n)]
    u0, j = np.random.rand(), 0
    for u in [(u0 + i) / n for i in range(n)]:
        while u > C[j]:
            j += 1
        indices.append(j - 1)
    return indices

def particle_filter(data, timestamps, N=1000):
    particles = np.zeros((N, 12))
    particles[:, :3] = data[['Latitude', 'Longitude', 'Speed']].values[0] + np.random.randn(N, 3)
    particles[:, 3:6] = 0  # Initial velocity
    particles[:, 6:9] = 0  # Initial acceleration
    particles[:, 9:12] = 0  # Initial gyro

    weights = np.ones(N) / N
    fused_data_pf = []

    for i in range(1, len(data)):
        dt = (timestamps[i] - timestamps[i - 1]).total_seconds()
        particles[:, :3] += particles[:, 3:6] * dt + 0.5 * data[['AccelX', 'AccelY', 'AccelZ']].values[i] * dt**2
        particles[:, 3:6] += data[['AccelX', 'AccelY', 'AccelZ']].values[i] * dt
        particles[:, 6:9] = data[['AccelX', 'AccelY', 'AccelZ']].values[i]
        particles[:, 9:12] = data[['GyroX', 'GyroY', 'GyroZ']].values[i]
        
        gps_measurement = data[['Latitude', 'Longitude', 'Speed']].values[i]
        if not np.isnan(gps_measurement).any():
            distances = np.linalg.norm(particles[:, :3] - gps_measurement, axis=1)
            weights *= scipy.stats.norm(0, 1).pdf(distances)
        
        weights += 1.e-300  # to avoid round-off to zero
        weights /= sum(weights)
        indices = resample(weights)
        particles[:] = particles[indices]
        weights.fill(1.0 / N)

        state_pf = np.average(particles, weights=weights, axis=0)
        fused_data_pf.append(state_pf)
    
    return pd.DataFrame(fused_data_pf, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'ax', 'ay', 'az', 'gx', 'gy', 'gz'])


In [ ]:
# Veri yükleme ve işleme
df = pd.read_csv('C:/Users/yigitc/Desktop/sensor_fusion/processed_data_with_timestamps.csv')
accel = df[['AccelX', 'AccelY', 'AccelZ']].values
gyro = df[['GyroX', 'GyroY', 'GyroZ']].values
gps = df[['Latitude', 'Longitude', 'Speed']].values
timestamps = pd.to_datetime(df['Timestamp'])

In [ ]:
# Normalize data
accel = (accel - np.mean(accel, axis=0)) / np.std(accel, axis=0)
gyro = (gyro - np.mean(gyro, axis=0)) / np.std(gyro, axis=0)

In [ ]:
# Kalman Filter (KF) tanımlama
kf = KalmanFilter(dim_x=12, dim_z=9)
kf.F = np.eye(12)
kf.H = np.zeros((9, 12))
kf.H[0:3, 0:3] = np.eye(3)    # GPS verileri için
kf.H[3:6, 6:9] = np.eye(3)    # İvmeölçer verileri için
kf.H[6:9, 9:12] = np.eye(3)   # Jiroskop verileri için
kf.P *= 0.1
kf.R = np.eye(9) * 0.5
kf.Q = np.eye(12) * 0.1

fused_data_kf = []
state = np.zeros(12)

for i in range(1, len(df)):
    dt = (timestamps[i] - timestamps[i - 1]).total_seconds()
    kf.F[0, 3] = kf.F[1, 4] = kf.F[2, 5] = dt
    kf.F[3, 6] = kf.F[4, 7] = kf.F[5, 8] = dt
    kf.F[6, 9] = kf.F[7, 10] = kf.F[8, 11] = dt
    kf.predict()
    kf.update(np.hstack((gps[i], accel[i], gyro[i])))
    fused_data_kf.append(kf.x.flatten())

fused_data_kf = pd.DataFrame(fused_data_kf, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'ax', 'ay', 'az', 'gx', 'gy', 'gz'])

In [ ]:
# UKF tanımlama
points = MerweScaledSigmaPoints(n=12, alpha=0.1, beta=2., kappa=-1)
ukf = UKF(dim_x=12, dim_z=9, fx=fx, hx=hx, dt=1, points=points)
ukf.x = np.zeros(12)
ukf.P *= 0.1
ukf.R = np.eye(9) * 0.5
ukf.Q = np.eye(12) * 0.1

fused_data_ukf = []
for i in range(1, len(df)):
    dt = (timestamps[i] - timestamps[i - 1]).total_seconds()
    ukf.predict(dt=dt)
    ukf.update(np.hstack((gps[i], accel[i], gyro[i])))
    fused_data_ukf.append(ukf.x.copy())

fused_data_ukf = pd.DataFrame(fused_data_ukf, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'ax', 'ay', 'az', 'gx', 'gy', 'gz'])

In [ ]:
# Particle Filter (PF) ile sonuçları hesapla
fused_data_pf = particle_filter(df, timestamps)

In [ ]:
# Filtrelerin sonuçlarını karşılaştırma
plt.figure(figsize=(15, 10))

plt.subplot(311)
plt.plot(fused_data_ekf['x'], label='EKF X Position')
plt.plot(fused_data_kf['x'], label='KF X Position')
plt.plot(fused_data_ukf['x'], label='UKF X Position')
plt.plot(fused_data_pf['x'], label='PF X Position')
plt.legend()
plt.title('X Position Comparison')

plt.subplot(312)
plt.plot(fused_data_ekf['y'], label='EKF Y Position')
plt.plot(fused_data_kf['y'], label='KF Y Position')
plt.plot(fused_data_ukf['y'], label='UKF Y Position')
plt.plot(fused_data_pf['y'], label='PF Y Position')
plt.legend()
plt.title('Y Position Comparison')

plt.subplot(313)
plt.plot(fused_data_ekf['z'], label='EKF Z Position')
plt.plot(fused_data_kf['z'], label='KF Z Position')
plt.plot(fused_data_ukf['z'], label='UKF Z Position')
plt.plot(fused_data_pf['z'], label='PF Z Position')
plt.legend()
plt.title('Z Position Comparison')

plt.tight_layout()
plt.show()

In [ ]:
#UKF nin seçilmesi
import pandas as pd
import numpy as np
from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.kalman import MerweScaledSigmaPoints
import matplotlib.pyplot as plt

# Örnek veri yükleme
df = pd.read_csv('processed_data_with_timestamps.csv')
timestamps = pd.to_datetime(df['Timestamp'])
gps_data = df[['Latitude', 'Longitude', 'Speed']].values
accel_data = df[['AccelX', 'AccelY', 'AccelZ']].values
gyro_data = df[['GyroX', 'GyroY', 'GyroZ']].values

# Verilerin normalize edilmesi
accel_data = (accel_data - np.mean(accel_data, axis=0)) / np.std(accel_data, axis=0)
gyro_data = (gyro_data - np.mean(gyro_data, axis=0)) / np.std(gyro_data, axis=0)

In [ ]:
# Durum geçiş modeli
def fx(state, dt):
    F = np.eye(12)
    F[0, 3] = F[1, 4] = F[2, 5] = dt
    F[3, 6] = F[4, 7] = F[5, 8] = dt
    F[6, 9] = F[7, 10] = F[8, 11] = dt
    return F @ state

# Ölçüm modeli
def hx(state):
     return np.hstack((state[:3], state[6:]))  # GPS, ivme ve jiroskop verilerini döndür

# Sigma noktalarını belirleme
points = MerweScaledSigmaPoints(n=12, alpha=0.1, beta=2., kappa=-1)

# UKF'yi tanımlama
ukf = UKF(dim_x=12, dim_z=9, fx=fx, hx=hx, dt=1, points=points)
ukf.x = np.zeros(12)
ukf.P *= 0.1
ukf.R = np.eye(9) * 0.5
ukf.Q = np.eye(12) * 0.05

In [ ]:
# Kovaryans matrisinin pozitif tanımlı kalmasını sağlamak için küçük bir değer ekleyelim
epsilon = 1e-6
ukf.P += np.eye(12) * epsilon

In [ ]:
# dt değerlerinin kontrol edilmesi
dt_values = (timestamps.diff().dropna()).apply(lambda x: x.total_seconds())
plt.plot(dt_values)
plt.title('Time Step (dt) Values Over Time')
plt.xlabel('Time Step Index')
plt.ylabel('dt (seconds)')
plt.show()

In [ ]:
fused_data_ukf = []

for i in range(1, len(df)):
    dt = (timestamps[i] - timestamps[i - 1]).total_seconds()
    ukf.predict(dt=dt)
    # GPS, ivmeölçer ve jiroskop verilerini kullanarak güncelleme
    measurement = np.hstack((gps_data[i], accel_data[i], gyro_data[i]))
    ukf.update(measurement)
    fused_data_ukf.append(ukf.x.copy())

fused_data_ukf = pd.DataFrame(fused_data_ukf, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'ax', 'ay', 'az', 'gx', 'gy', 'gz'])

In [ ]:
plt.figure(figsize=(15, 10))

plt.subplot(311)
plt.plot(fused_data_ukf['x'], label='UKF X Position')
plt.legend()
plt.title('X Position Over Time')

plt.subplot(312)
plt.plot(fused_data_ukf['y'], label='UKF Y Position')
plt.legend()
plt.title('Y Position Over Time')

plt.subplot(313)
plt.plot(fused_data_ukf['z'], label='UKF Z Position')
plt.legend()
plt.title('Z Position Over Time')

plt.tight_layout()
plt.show()

In [ ]:
from scipy import stats

# Örnek Z-skore yöntemi ile aykırı değer tespiti
z_scores = np.abs(stats.zscore(df[['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ']]))
outliers = (z_scores > 3).any(axis=1)
df_cleaned = df[~outliers].copy()  # Uyarıları engellemek için kopyalama

# Eksik değerleri kontrol etme ve doldurma
df_cleaned = df_cleaned.infer_objects(copy=False)  # Veri tiplerini ayarlama
numeric_columns = df_cleaned.select_dtypes(include=[np.number]).columns
df_cleaned[numeric_columns] = df_cleaned[numeric_columns].interpolate(method='linear', inplace=False)  # Lineer interpolasyon

In [ ]:
# Eksik değerleri kontrol etme ve doldurma
df.ffill(inplace=True)  # İleri doldurma
df.bfill(inplace=True)  # Geri doldurma

In [ ]:
# Gerçek ölçümler ve filtre sonuçları arasındaki hataları hesaplama
# Zaman damgalarını karşılaştırarak senkronize etme
common_timestamps = timestamps[1:][~outliers[1:]].reset_index(drop=True)
fused_data_ukf['Timestamp'] = common_timestamps

# Senkronize veri çerçeveleri oluşturma
sync_df_cleaned = df_cleaned[['Latitude', 'Longitude', 'Speed']].iloc[1:].reset_index(drop=True)
sync_fused_data_ukf = fused_data_ukf[['x', 'y', 'z']].iloc[:len(sync_df_cleaned)]

# Hataları hesaplama
errors = sync_df_cleaned.values - sync_fused_data_ukf.values
mse = np.mean(errors**2, axis=0)
print(f"Mean Squared Error: {mse}")

# Kestirim sonuçlarının varyansını hesaplama
variance = fused_data_ukf[['x', 'y', 'z']].var()
print(f"Variance: {variance}")

In [ ]:
#Sürücü Davranışı Modellemesi

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Örnek veri yükleme
df = pd.read_csv('processed_data_with_timestamps.csv')

# Zaman damgalarını al
timestamps = pd.to_datetime(df['Timestamp'])

# GPS, ivme ve jiroskop verilerini al
gps_data = df[['Latitude', 'Longitude', 'Speed']].values
accel_data = df[['AccelX', 'AccelY', 'AccelZ']].values
gyro_data = df[['GyroX', 'GyroY', 'GyroZ']].values

# İvme büyüklüğünü hesapla
df['Accel_Magnitude'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

# Özelliklerin hesaplanması (örnek olarak pencere boyutu 10)
window_size = 10

df['Accel_Mean'] = df['Accel_Magnitude'].rolling(window=window_size).mean()
df['Accel_Std'] = df['Accel_Magnitude'].rolling(window=window_size).std()
df['Accel_Max'] = df['Accel_Magnitude'].rolling(window=window_size).max()
df['Accel_Min'] = df['Accel_Magnitude'].rolling(window=window_size).min()

df['Gyro_Mean'] = np.sqrt(df['GyroX']**2 + df['GyroY']**2 + df['GyroZ']**2).rolling(window=window_size).mean()
df['Gyro_Std'] = np.sqrt(df['GyroX']**2 + df['GyroY']**2 + df['GyroZ']**2).rolling(window=window_size).std()
df['Gyro_Max'] = np.sqrt(df['GyroX']**2 + df['GyroY']**2 + df['GyroZ']**2).rolling(window=window_size).max()
df['Gyro_Min'] = np.sqrt(df['GyroX']**2 + df['GyroY']**2 + df['GyroZ']**2).rolling(window=window_size).min()

df['Speed_Mean'] = df['Speed'].rolling(window=window_size).mean()
df['Speed_Std'] = df['Speed'].rolling(window=window_size).std()
df['Speed_Max'] = df['Speed'].rolling(window=window_size).max()
df['Speed_Min'] = df['Speed'].rolling(window=window_size).min()

# Etiket oluşturma (basit eşik tabanlı)
def create_labels(row):
    if row['Accel_Magnitude'] > 15 or row['Speed'] > 50:
        return 'Aggressive'
    elif row['Accel_Magnitude'] < 5 and row['Speed'] < 30:
        return 'Careful'
    else:
        return 'Normal'

df['Label'] = df.apply(create_labels, axis=1)

# Özellikler ve etiketlerin temizlenmesi
features = df[['Accel_Mean', 'Accel_Std', 'Accel_Max', 'Accel_Min', 
               'Gyro_Mean', 'Gyro_Std', 'Gyro_Max', 'Gyro_Min',
               'Speed_Mean', 'Speed_Std', 'Speed_Max', 'Speed_Min']]
labels = df['Label']

# Hem özellikler hem de etiketlerdeki eksik değerleri aynı anda düşür
features_labels = pd.concat([features, labels], axis=1).dropna()

# Temizlenmiş özellikler ve etiketler
features = features_labels.iloc[:, :-1]
labels = features_labels.iloc[:, -1]

# Eğitim ve test verilerine ayırma
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Model eğitimi
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Model değerlendirme
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Sonuçları görselleştirme
plt.figure(figsize=(15, 5))
plt.plot(timestamps, df['Accel_Magnitude'], label='Accel Magnitude')
plt.scatter(timestamps, df['Label'], c='red', label='Label', alpha=0.5)
plt.xlabel('Time')
plt.ylabel('Accel Magnitude')
plt.legend()
plt.show()

In [ ]:
#pip install pandas numpy matplotlib


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
# Set the animation embed limit to a larger value (in MB)
plt.rcParams['animation.embed_limit'] = 200  # Increase to 50 MB or more as needed

# Enable inline plotting
%matplotlib inline


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Load and process data from .txt file
data_path = 'C:/Users/yigitc/Desktop/CoolTerm Capture (Untitled_0.stc) 2024-05-03 12-16-51-551.txt'

with open(data_path, 'r') as file:
    lines = file.readlines()

processed_data = []
temp_data = {}

for line in lines:
    line = line.strip()
    if 'Latitude:' in line:
        temp_data['Latitude'] = float(line.split(':')[1].strip())
    elif 'Longitude:' in line:
        temp_data['Longitude'] = float(line.split(':')[1].strip())
    elif 'Speed (km/s):' in line:
        speed = float(line.split(':')[1].strip())
        temp_data['Speed'] = speed if 0 <= speed <= 100 else np.nan  # Assuming 0 to 100 km/s is reasonable
    elif 'AccelX:' in line:
        parts = line.split(',')
        temp_data['AccelX'] = float(parts[0].split(':')[1].strip())
        temp_data['AccelY'] = float(parts[1].split(':')[1].strip())
        temp_data['AccelZ'] = float(parts[2].split(':')[1].strip())
        temp_data['GyroX'] = float(parts[3].split(':')[1].strip())
        temp_data['GyroY'] = float(parts[4].split(':')[1].strip())
        temp_data['GyroZ'] = float(parts[5].split(':')[1].strip())
        if 'Latitude' in temp_data and 'Longitude' in temp_data and 'Speed' in temp_data:
            processed_data.append(temp_data.copy())

df = pd.DataFrame(processed_data)

# Filter out rows with NaN values
df.dropna(inplace=True)

# Create a timestamp column
start_datetime = pd.to_datetime('2024-05-03 12:16:51.551')
df['Timestamp'] = [start_datetime + pd.to_timedelta(1 * i, unit='s') for i in range(len(df))]

# Compute Accel_Magnitude
df['Accel_Magnitude'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

# Create a kinetic energy column (example, adjust as needed)
m = 6500  # mass in kg
df['Speed_m/s'] = df['Speed'] * 1000 / 3600
df['Kinetic_Energy'] = 0.5 * m * df['Speed_m/s']**2
df['Total_Acceleration'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

# Label creation for simulation purposes
def create_labels(row):
    if row['Accel_Magnitude'] > 15 or row['Speed'] > 50:
        return 'Aggressive'
    elif row['Accel_Magnitude'] < 5 and row['Speed'] < 30:
        return 'Careful'
    else:
        return 'Normal'

df['Label'] = df.apply(create_labels, axis=1)

# Convert Timestamp to Unix timestamp once
df['Unix_Timestamp'] = df['Timestamp'].astype('int64') / 10**9

# Plotting setup for Kinetic Energy, Total Acceleration, and GPS data
fig, axs = plt.subplots(2, 2, figsize=(20, 15))  # Adjust figsize for larger display
fig.tight_layout(pad=5.0)

# Kinetic Energy plot
ax1 = axs[0, 0]
ax1.set_title('Kinetic Energy Over Time')
ax1.set_xlabel('Time')
ax1.set_ylabel('Kinetic Energy')
ax1.set_ylim(0, 500000)  # Manually set y-axis limit
line1, = ax1.plot([], [], label='Kinetic Energy')
ax1.legend()

# Total Acceleration plot
ax2 = axs[0, 1]
ax2.set_title('Total Acceleration Over Time')
ax2.set_xlabel('Time')
ax2.set_ylabel('Acceleration (m/s^2)')
ax2.set_ylim(0, 20)  # Manually set y-axis limit
line2, = ax2.plot([], [], label='Total Acceleration', color='red')
ax2.legend()

# GPS plot
ax3 = axs[1, 0]
ax3.set_title('GPS Coordinates Over Time')
ax3.set_xlabel('Longitude')
ax3.set_ylabel('Latitude')
line3, = ax3.plot([], [], label='GPS Coordinates', color='green')
ax3.legend()

# Accel Magnitude and Labels plot
ax4 = axs[1, 1]
ax4.set_title('Accel Magnitude and Labels Over Time')
ax4.set_xlabel('Time')
ax4.set_ylabel('Accel Magnitude')
ax4.set_ylim(0, 20)  # Manually set y-axis limit
line4, = ax4.plot([], [], label='Accel Magnitude')
sc = ax4.scatter([], [], color='red', label='Label')
ax4.legend()

# Add text annotation for displaying GPS data
text_annotation = ax3.text(0.05, 0.95, '', transform=ax3.transAxes, fontsize=12, verticalalignment='top')

# Combined animation function
def animate(i):
    step = max(1, int(len(df) / 250))  # Adjust step size based on total length of data

    # Update Kinetic Energy and Total Acceleration
    xdata = df['Unix_Timestamp'][:i*step]
    ydata_kinetic = df['Kinetic_Energy'][:i*step]
    ydata_accel = df['Total_Acceleration'][:i*step]

    line1.set_data(xdata, ydata_kinetic)
    line2.set_data(xdata, ydata_accel)

    ax1.relim()
    ax1.autoscale_view()
    ax2.relim()
    ax2.autoscale_view()

    # Update GPS data and text annotation
    lon_data = df['Longitude'][:i*step]
    lat_data = df['Latitude'][:i*step]

    line3.set_data(lon_data, lat_data)

    # Update text annotation with current GPS data
    if i * step < len(df):
        current_data = df.iloc[i * step]
        text_annotation.set_text(f"Latitude: {current_data['Latitude']:.6f}\nLongitude: {current_data['Longitude']:.6f}\nSpeed (km/s): {current_data['Speed']:.2f}")

    ax3.relim()
    ax3.autoscale_view()

    # Update Accel Magnitude and Labels
    ydata = df['Accel_Magnitude'][:i*step]

    line4.set_data(xdata, ydata)

    # Update scatter plot for labels
    labels = df['Label'][:i*step]
    colors = ['red' if label == 'Aggressive' else 'blue' for label in labels]
    sc.set_offsets(np.c_[xdata, ydata])
    sc.set_color(colors)

    ax4.relim()
    ax4.autoscale_view()

    return line1, line2, line3, line4, sc, text_annotation

# Create animation
ani = animation.FuncAnimation(fig, animate, frames=int(len(df) / 10), interval=100, blit=True)  # Increased interval to 100ms

# Display animation
plt.close(fig)  # Close the figure to avoid displaying a static version
HTML(ani.to_jshtml())


In [ ]:
# Load data
df = pd.read_csv('processed_data_with_timestamps.csv')
timestamps = pd.to_datetime(df['Timestamp'])

# Compute Accel_Magnitude
df['Accel_Magnitude'] = np.sqrt(df['AccelX']**2 + df['AccelY']**2 + df['AccelZ']**2)

# Label creation for simulation purposes
def create_labels(row):
    if row['Accel_Magnitude'] > 15 or row['Speed'] > 50:
        return 'Aggressive'
    elif row['Accel_Magnitude'] < 5 and row['Speed'] < 30:
        return 'Careful'
    else:
        return 'Normal'
df['Label'] = df.apply(create_labels, axis=1)


In [ ]:
# Function to create the animation for Kinetic Energy and Total Acceleration
def animate_kinetic_accel(i):
    xdata = timestamps[:i+1].astype('int64') / 10**9  # Convert to Unix timestamp
    ydata_kinetic = df['Kinetic_Energy'][:i+1]
    ydata_accel = df['Total_Acceleration'][:i+1]

    line1.set_data(xdata, ydata_kinetic)
    line2.set_data(xdata, ydata_accel)

    ax1.relim()
    ax1.autoscale_view()
    ax2.relim()
    ax2.autoscale_view()

    return line1, line2

# Function to create the animation for Accel Magnitude and Labels
def animate_driver_behavior(i):
    xdata = timestamps[:i+1].astype('int64') / 10**9  # Convert to Unix timestamp
    ydata = df['Accel_Magnitude'][:i+1]
    
    line3.set_data(xdata, ydata)
    
    # Update scatter plot for labels
    labels = df['Label'][:i+1]
    colors = ['red' if label == 'Aggressive' else 'blue' for label in labels]
    sc.set_offsets(np.c_[xdata, ydata])
    sc.set_color(colors)
    
    ax3.relim()
    ax3.autoscale_view()
    
    return line3, sc

# Plotting setup for Kinetic Energy and Total Acceleration
fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.set_title('Kinetic Energy Over Time')
ax1.set_xlabel('Time')
ax1.set_ylabel('Kinetic Energy')
line1, = ax1.plot([], [], label='Kinetic Energy')
ax1.legend()

ax2.set_title('Total Acceleration Over Time')
ax2.set_xlabel('Time')
ax2.set_ylabel('Acceleration (m/s^2)')
line2, = ax2.plot([], [], label='Total Acceleration', color='red')
ax2.legend()

ani1 = animation.FuncAnimation(fig1, animate_kinetic_accel, frames=len(df), interval=100, blit=True)
HTML(ani1.to_jshtml())

# Plotting setup for Accel Magnitude and Labels
fig2, ax3 = plt.subplots(figsize=(10, 5))
ax3.set_title('Accel Magnitude and Labels Over Time')
ax3.set_xlabel('Time')
ax3.set_ylabel('Accel Magnitude')
line3, = ax3.plot([], [], label='Accel Magnitude')
sc = ax3.scatter([], [], color='red', label='Label')
ax3.legend()

ani2 = animation.FuncAnimation(fig2, animate_driver_behavior, frames=len(df), interval=100, blit=True)
HTML(ani2.to_jshtml())
